In [3]:
import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

convert original imgs to grayscale, resize, and flatten:

In [4]:
train_input_folder_wildfire = '/Users/chasecoogan/Documents/bu/model/datasets/wildfire/train/wildfire'
train_input_folder_no_wildfire = '/Users/chasecoogan/Documents/bu/model/datasets/wildfire/train/nowildfire'

test_input_folder_wildfire = '/Users/chasecoogan/Documents/bu/model/datasets/wildfire/test/wildfire'
test_input_folder_no_wildfire = '/Users/chasecoogan/Documents/bu/model/datasets/wildfire/test/nowildfire'

image_size = (128, 128)

def convert_img_to_greyscale(directory, label):
    images = []
    labels = []

    for ea in os.listdir(directory):
        if ea.endswith(".jpg"):
            path = os.path.join(directory, ea)

            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, image_size)

            img = img.astype('float32') / 255

            images.append(img)
            labels.append(label)
    return images, labels


train_images_wildfire, train_labels_wildfire = convert_img_to_greyscale(train_input_folder_wildfire, label=1)
train_images_no_wildfire, train_labels_no_wildfire = convert_img_to_greyscale(train_input_folder_no_wildfire, label=0)

test_images_wildfire, test_labels_wildfire = convert_img_to_greyscale(test_input_folder_wildfire, label=1)
test_images_no_wildfire, test_labels_no_wildfire = convert_img_to_greyscale(test_input_folder_no_wildfire, label=0)

train_images = np.array(train_images_wildfire + train_images_no_wildfire)
train_labels = np.array(train_labels_wildfire + train_labels_no_wildfire)

test_images = np.array(test_images_wildfire + test_images_no_wildfire)
test_labels = np.array(test_labels_wildfire + test_labels_no_wildfire)

train_images = train_images.reshape(train_images.shape[0], 128, 128, 1)
test_images = test_images.reshape(test_images.shape[0], 128, 128, 1)


# --------- skip flatten and pca for now ---------- #

# images = images.reshape(images.shape[0], -1)

# n_components = 100
# pca = PCA(n_components=n_components)
# pca_images = pca.fit_transform(images)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Accuracy:", test_accuracy)


Premature end of JPEG file
Premature end of JPEG file
/Users/chasecoogan/Documents/bu/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 184s 193ms/step - accuracy: 0.8572 - loss: 0.3197 - val_accuracy: 0.9213 - val_loss: 0.2365
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 181s 192ms/step - accuracy: 0.9208 - loss: 0.2163 - val_accuracy: 0.9414 - val_loss: 0.1616
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 170s 179ms/step - accuracy: 0.9291 - loss: 0.1960 - val_accuracy: 0.9479 - val_loss: 0.1527
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 170s 180ms/step - accuracy: 0.9372 - loss: 0.1734 - val_accuracy: 0.9575 - val_loss: 0.1321
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 163s 172ms/step - accuracy: 0.9421 - loss: 0.1605 - val_accuracy: 0.9592 - val_loss: 0.1125
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 162s 172ms/step - accuracy: 0.9454 - loss: 0.1471 - val_accuracy: 0.9263 - val_loss: 0.2433
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 162s 172ms/step - accuracy: 0.9498 - loss: 0.1346 - val_accuracy: 0.9613 - val_loss: 0.1144
Epoch 8/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 167s 177ms/step - accuracy: 0.9600 -

In [ ]:
# save model

model.save('wildfire_classifier_model.h5')
print('saved')

NameError: name 'model' is not defined

In [33]:
def predict_img_class(model, img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    img = cv2.resize(img, (128, 128))
    img = img.astype('float32') / 255.0
    img = np.reshape(img, (1, 128, 128, 1))
    prediction = model.predict(img)

    if prediction[0] > 0.5:
        return "Wildfire"
    else:
        return "Not Wildfire"
    

## example use ##
path = '/Users/chasecoogan/Documents/bu/model/datasets/wildfire/valid/wildfire/-57.3633,51.4978.jpg'
result = predict_img_class(model, path)
print(f'path: {path} with res: {result}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
path: /Users/chasecoogan/Documents/bu/model/datasets/wildfire/valid/wildfire/-57.3633,51.4978.jpg with res: Not Wildfire
